# Task

Given two sentences that are known to be paraphrases, pick the phrases that are similar and
contribute to their overall similarity

# Example:

Sentence 1: Charlie Chan is off the case for the Fox Movie Channel.  <br/>
Sentence 2: The Fox Movie Channel has banned Charlie Chan

Output: <br/>

Banned, off the case <br/>
Charlie Chan, Charlie Chan <br/>
Fox Movie Channel, Fox Movie Channel<br/>

# Import Libraries

In [742]:
import numpy as np
import os, sys
from sklearn.metrics.pairwise import cosine_similarity
from bert_serving.client import BertClient
import spacy
import textacy
from spacy import displacy
import en_core_web_sm
import math

# Load Language Models

Spacy provides downloadable pretrained language models for important natural language tasks such as tagging, parsing and entity recognition. 

A comphrensive list of available models can be seen at https://spacy.io/models/en. 

For the sake of this task, we are using **en_core_web_sm**. The following code expects this module is already available. If not, it can be downloaded by running
```
python -m spacy download en_core_web_sm
```

Models can be loaded using spaCy's build-in loader, or as a normal python module.

In [743]:
# nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()

# Load sentences that are knwon to be paraphrases using the language model

In [744]:
sentence_1 = nlp("Feelings about current business conditions improved substantially from the first quarter, jumping from 40 to 55.")
sentence_2 = nlp("Assessment of current business conditions improved substantially, the Conference Board said, jumping to 55 from 40 in the first quarter.")

# Possible techniques to solve the problem

The task can be solved by decomposing the input sentences into individual phrases and comparing similarity of the obtained phrases in their vector representation.

## Obtaining phrases

- **Method 1: Chunking**

    Text chunking, also referred to as shallow parsing, is a task that follows Part-Of-Speech Tagging and that adds more structure to the sentence. The result is a grouping of the words in “chunks”. We use this technique for noun phrase extraction.
    
- **Method 2: Parsing**
    Parsing in NLP is the process of determining the syntactic structure of a text by analyzing its constituent words based on an underlying grammar (of the language). Parsing results in a tree and can be of following two types
    - Constituent parsing
    - Dependency parsing

## Calculating similarity
Similarity between phrases can be calculated usign any distance metric technique (like Jaccard similarity, or Cosine scores). For this, the two phrases being compared needs to be converted into their respective vector form. One of the popular ways to convert words into numeric vectors is using distributed representation of words. However, context-free models such as word2vec or GloVe generate a single word embedding representation for each word in the vocabulary and do not capture polysemy. Contextual models like BERT and ELMo generate a representation of each word that is based on the other words in the sentence. 


### Current Approach

For the sake of this assignment, we have used **Method 1** mentioned above to extract the noun-phrases from input sentence pairs. This is achieved using spaCy. For similarity comparison, we use pre-trained BERT embeddings that run through bert-as-a-service (https://github.com/hanxiao/bert-as-service). Cosine similarity is obtained by simply calculating the dot product of the vectors.

In [745]:
# This function extracts the noun phrases from a given input sentence and returns a list
def extract_noun_phrases(doc):
    phrases_extracted = []
    for chunk in doc.noun_chunks:
#         print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)
        phrases_extracted.append(chunk.text)
    return phrases_extracted

In [746]:
sentence_1_phrases = extract_noun_phrases(sentence_1)
sentence_1_phrases

['Feelings', 'current business conditions', 'the first quarter']

In [747]:
sentence_2_phrases = extract_noun_phrases(sentence_2)
sentence_2_phrases

['Assessment',
 'current business conditions',
 'the Conference Board',
 'the first quarter']

### Following code assumes that a separate bert-serving-server is running at port 5555, that converts a string into BERT encoding

```
bert-serving-start -model_dir /Users/nalinc/Projects/word_embeddings/bert/uncased_L-12_H-768_A-12/ -num_worker=4
```

In [748]:
def find_similar_phrases(sentence_1_phrases, sentence_2_phrases):
    def calculate_similarity(pair):
        query_vec_1, query_vec_2 = bert_client.encode(pair)
        cosine = np.dot(query_vec_1, query_vec_2) / (np.linalg.norm(query_vec_1) * np.linalg.norm(query_vec_2))
        return 1/(1 + math.exp(-100*(cosine - 0.95)))
    
    similar_phrases = []
    # Open a new connection to BERT Server
    with BertClient(port=5555, port_out=5556, check_version=False) as bert_client:
        # Find which of the two list of phrases is longer. We should only compare phrases with respect to smaller sentence
        longer_phrase_length = sentence_1_phrases if len(sentence_1_phrases) > len(sentence_2_phrases) else sentence_2_phrases
        shorter_phrase_length = sentence_1_phrases if len(sentence_1_phrases) <= len(sentence_2_phrases) else sentence_2_phrases

        for i in shorter_phrase_length:
            most_similar_j_index, most_similar_j = 0,0
            for index_j,j in enumerate(longer_phrase_length):
                print(i,"\t\t\t",j)
                i_j_similarity = calculate_similarity([i,j])
                if i_j_similarity >= most_similar_j:
                    most_similar_j_index = index_j
                    most_similar_j = i_j_similarity
            print(most_similar_j)
            similar_phrases.append([i, longer_phrase_length[most_similar_j_index]])
    return similar_phrases
        

In [749]:
similar_phrases = find_similar_phrases(sentence_1_phrases, sentence_2_phrases)

Feelings 			 Assessment
Feelings 			 current business conditions
Feelings 			 the Conference Board
Feelings 			 the first quarter
8.828242445860665e-07
current business conditions 			 Assessment
current business conditions 			 current business conditions
current business conditions 			 the Conference Board
current business conditions 			 the first quarter
0.9933072283262605
the first quarter 			 Assessment
the first quarter 			 current business conditions
the first quarter 			 the Conference Board
the first quarter 			 the first quarter
0.9933071490757153


# Final Output

In [751]:
# Simply output the phrases with maximum similarity score in above output
similar_phrases

[['Feelings', 'Assessment'],
 ['current business conditions', 'current business conditions'],
 ['the first quarter', 'the first quarter']]

## Output with other examples


In [752]:
sentence_1 = nlp("Charlie Chan is off the case for the Fox Movie Channel.")
sentence_2 = nlp("The Fox Movie Channel has banned Charlie Chan.")

sentence_1_phrases = extract_noun_phrases(sentence_1)
sentence_2_phrases = extract_noun_phrases(sentence_2)

similar_phrases = find_similar_phrases(sentence_1_phrases, sentence_2_phrases)
similar_phrases

The Fox Movie Channel 			 Charlie Chan
The Fox Movie Channel 			 the case
The Fox Movie Channel 			 the Fox Movie Channel
0.9933071490757153
Charlie Chan 			 Charlie Chan
Charlie Chan 			 the case
Charlie Chan 			 the Fox Movie Channel
0.9933070698242378


[['The Fox Movie Channel', 'the Fox Movie Channel'],
 ['Charlie Chan', 'Charlie Chan']]

# Alternative Way(s)

Results can be improved and further fine-tuned by using various techniques as described below
- Using other rule-based approaches to extract verb-phrases, adjective-phrases etc.
- By training a deep neural network on various dependency-trees obtained from the pair of sentences. Through this, the problem can be decomposed to 

Essentially, if we can extract more relevant (as well as consistent) phrases from the input sentence pairs, then we can form more such similar pair of phrases. Following code illustrates a proof-of-concept for this kind of approach.

### Visualizing the dependency tree of input sentences

In [753]:
displacy.render(sentence_1, style="dep")
displacy.render(sentence_2, style="dep")

In [754]:
def extract_phrases(doc):
    phrases_extracted = []
    # print([token.text for token in doc[2].lefts])
    root = [token for token in doc if token.head == token][0]
    left_subtree_subjects = list(root.lefts)
    right_subtree_subjects = list(root.rights)
    subjects = left_subtree_subjects + right_subtree_subjects
    
    for subject in left_subtree_subjects:
        _phrases = []
        for descendant in subject.subtree:
            if descendant.dep_ != "punct":
                _phrases.append(descendant.text)
            print(descendant.text, descendant.dep_, descendant.n_lefts, descendant.n_rights, [ancestor.text for ancestor in descendant.ancestors])
        print("--")
        if len(_phrases):
            phrases_extracted.append(" ".join(_phrases))

    phrases_extracted.append("".join([root.text]))
    
    for subject in right_subtree_subjects:
        _phrases = []
        for descendant in subject.subtree:
            if descendant.dep_ != "punct":
                _phrases.append(descendant.text)
            print(descendant.text, descendant.dep_, descendant.n_lefts, descendant.n_rights, [ancestor.text for ancestor in descendant.ancestors])
        print("--")
        if len(_phrases):
            phrases_extracted.append(" ".join(_phrases))
    
    return phrases_extracted

In [755]:
sentence_1_phrases = extract_phrases(sentence_1)

Charlie compound 0 0 ['Chan', 'is']
Chan nsubj 1 0 ['is']
--
off prep 0 1 ['is']
the det 0 0 ['case', 'off', 'is']
case pobj 1 0 ['off', 'is']
--
for prep 0 1 ['is']
the det 0 0 ['Channel', 'for', 'is']
Fox compound 0 0 ['Movie', 'Channel', 'for', 'is']
Movie compound 1 0 ['Channel', 'for', 'is']
Channel pobj 2 0 ['for', 'is']
--
. punct 0 0 ['is']
--


In [756]:
sentence_2_phrases = extract_phrases(sentence_2)

The det 0 0 ['Channel', 'banned']
Fox compound 0 0 ['Channel', 'banned']
Movie compound 0 0 ['Channel', 'banned']
Channel nsubj 3 0 ['banned']
--
has aux 0 0 ['banned']
--
Charlie compound 0 0 ['Chan', 'banned']
Chan dobj 1 0 ['banned']
--
. punct 0 0 ['banned']
--


In [757]:
find_similar_phrases(sentence_1_phrases, sentence_2_phrases)

Charlie Chan 			 The Fox Movie Channel
Charlie Chan 			 has
Charlie Chan 			 banned
Charlie Chan 			 Charlie Chan
0.9933070698242378
is 			 The Fox Movie Channel
is 			 has
is 			 banned
is 			 Charlie Chan
0.02589186437030154
off the case 			 The Fox Movie Channel
off the case 			 has
off the case 			 banned
off the case 			 Charlie Chan
4.4969501981066854e-13
for the Fox Movie Channel 			 The Fox Movie Channel
for the Fox Movie Channel 			 has
for the Fox Movie Channel 			 banned
for the Fox Movie Channel 			 Charlie Chan
0.003524340980437101


[['Charlie Chan', 'Charlie Chan'],
 ['is', 'has'],
 ['off the case', 'banned'],
 ['for the Fox Movie Channel', 'The Fox Movie Channel']]

# References

1. https://spacy.io/usage/linguistic-features
2. https://github.com/google-research/bert#pre-trained-models
3. https://bert-as-service.readthedocs.io/en/latest/#
